In [1]:
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch
import torchvision.transforms as transforms
import torch.optim as optim

In [2]:
class ResNet(nn.Module):
    def __init__(self, n=2, r=10):
        
        super().__init__()
        self.n = n
        self.r = r
        
        self.red0 = nn.Conv2d(3, 16, kernel_size=1, stride=1)
        l1 = []
        for i in range(2 * n):
            if i == 0:
                l1.append(nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1))
            else:
                l1.append(nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1))
        self.l1 = nn.ModuleList(l1)
        
        self.red1 = nn.Conv2d(16, 32, kernel_size=1, stride=2)
        
        l2 = []
        for i in range(2 * n):
            if i == 0:
                l2.append(nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1))
            else:
                l2.append(nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1))
        
        self.l2 = nn.ModuleList(l2)
        
        self.red2 = nn.Conv2d(32, 64, kernel_size=1, stride=2)
        l3 = []
        for i in range(2 * n):
            if i == 0:
                l3.append(nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1))
            else:
                l3.append(nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1))
        
        self.l3 = nn.ModuleList(l3)
        self.fc = nn.Linear(4096, 10)
        
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, X):
        
        i1 = [X]
        z1 = [None]
        o1 = [None]
        
        for i in range(1, self.n * 2 + 1):
            z = self.l1[i - 1](i1[i - 1])
            z1.append(z)
            o1.append(F.relu(z))
            
            if i % 2 == 0:
                if i == 2:
                    i1.append(o1[i] + self.red0(i1[i - 2]))
                else:
                    i1.append(o1[i] + i1[i - 2])
            else:
                i1.append(o1[i])
        
        
        i2 = [o1[self.n * 2]]
        z2 = [None]
        o2 = [None]
        
        for i in range(1, self.n * 2 + 1):
            z = self.l2[i - 1](i2[i - 1])
            z2.append(z)
            o2.append(F.relu(z))
            
            if i % 2 == 0:
                if i == 2:
                    i2.append(o2[i] + self.red1(i2[i - 2]))
                else:
                    i2.append(o2[i] + i2[i - 2])
            else:
                i2.append(o2[i])
                    
        
        i3 = [o2[self.n * 2]]
        z3 = [None]
        o3 = [None]
        
        for i in range(1, self.n * 2 + 1):
            z = self.l3[i - 1](i3[i - 1])
            z3.append(z)
            o3.append(F.relu(z))
            
            if i % 2 == 0:
                if i == 2:
                    i3.append(o3[i] + self.red2(i3[i - 2]))
                else:
                    i3.append(o3[i] + i3[i - 2])
            else:
                i3.append(o3[i])
        
        output = i3[self.n * 2].flatten(1)
        output = self.fc(output)
        #print(output)
        return F.softmax(output, dim=1)


In [7]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='/home/cse/dual/cs5180404/scratch/col870/', train=True, transform=transform)
print(len(trainset))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=4)

50000


In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
net = ResNet().to(device)
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=3e-6)
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        #print(inputs.shape)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        #print(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        #print(loss.item())
        if i % 50 == 49:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        

[1,    50] loss: 0.040466
[1,   100] loss: 0.041181
[1,   150] loss: 0.041827
[1,   200] loss: 0.040790


In [23]:
from sklearn.metrics import r2_score, precision_score, recall_score, f1_score, confusion_matrix

testset = torchvision.datasets.CIFAR10(root='/home/cse/dual/cs5180404/scratch/col870/', train=True, transform=transform)
print(len(testset))
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=4)

actual, prediction = [], []
for i, data in enumerate(testloader, 0):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    #print(inputs.shape)
    actual.extend(labels.squeeze().tolist())
    
    optimizer.zero_grad()
    output = net(inputs)
    
    prediction.extend(torch.argmax(output, dim=1).squeeze().tolist())
    #print(output)
print(f1_score(actual, prediction, average='macro'))
print(confusion_matrix(actual, prediction))

50000
0.8227947172270248
[[4298   68  101   77   54   21   23   57  179  122]
 [  38 4577   23   28   18   17   34   18   66  181]
 [ 230   37 3574  241  252  192  248  100   74   52]
 [  79   23  224 3583  192  436  221  131   50   61]
 [ 108   21  183  189 3928  142  156  200   35   38]
 [  34   22  175  505  206 3647  124  194   25   68]
 [  33   32  147  164  113   86 4353   26   20   26]
 [  41   20  105  157  164  168   34 4247   15   49]
 [ 136   79   50   46   27   19   20    7 4554   62]
 [  96  214   25   57   34   28   28   43   60 4415]]
